In [1]:
import pandas as pd
df = pd.read_parquet("D:\\ME\\sumanth\\OUBT\\code\\data\\yellow_tripdata_2025-08.parquet")
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3574091 entries, 0 to 3574090
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [2]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,2,2025-08-01 00:52:23,2025-08-01 01:12:20,1.0,8.44,1.0,N,138,141,1,33.8,6.0,0.5,5.00,6.94,1.0,57.49,2.5,1.75,0.00
1,2,2025-08-01 00:03:01,2025-08-01 00:15:33,2.0,4.98,1.0,N,138,193,1,21.2,6.0,0.5,0.00,0.00,1.0,30.45,0.0,1.75,0.00
2,7,2025-08-01 00:24:38,2025-08-01 00:24:38,2.0,1.89,1.0,N,249,45,1,14.2,0.0,0.5,3.99,0.00,1.0,23.94,2.5,0.00,0.75
3,7,2025-08-01 00:48:19,2025-08-01 00:48:19,1.0,2.35,1.0,N,79,229,1,11.4,0.0,0.5,3.43,0.00,1.0,20.58,2.5,0.00,0.75
4,2,2025-08-01 00:25:34,2025-08-01 00:33:18,1.0,2.14,1.0,N,43,48,1,11.4,1.0,0.5,2.57,0.00,1.0,19.72,2.5,0.00,0.75


In [3]:
import re
import boto3
import pandas as pd
import pyarrow.parquet as pq
from io import BytesIO

# ----------------------------
# Config
# ----------------------------
BUCKET = "new-demo-bucket-sumenth-12"
PREFIX = "audit/day10_run_summary/"  # must end with /

# If your AWS CLI works, boto3 will usually pick creds automatically.
# Otherwise set AWS_PROFILE or env vars (AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_SESSION_TOKEN)
session = boto3.Session()  # or boto3.Session(profile_name="default")
s3 = session.client("s3")

def list_keys(bucket: str, prefix: str) -> list[str]:
    keys = []
    paginator = s3.get_paginator("list_objects_v2")
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get("Contents", []):
            keys.append(obj["Key"])
    return keys

def read_parquet_keys_to_df(bucket: str, keys: list[str]) -> pd.DataFrame:
    dfs = []
    for key in keys:
        # Read only the parquet data files (skip delta logs and folder markers)
        if not key.endswith(".parquet"):
            continue
        if "/_delta_log/" in key or key.endswith("_delta_log"):
            continue

        print(f"Reading: s3://{bucket}/{key}")
        bio = BytesIO()
        s3.download_fileobj(bucket, key, bio)
        bio.seek(0)

        table = pq.read_table(bio)
        dfs.append(table.to_pandas())

    if not dfs:
        raise RuntimeError("No parquet data files found under prefix (excluding _delta_log).")

    return pd.concat(dfs, ignore_index=True)

def main():
    keys = list_keys(BUCKET, PREFIX)
    print(f"Found {len(keys)} objects under s3://{BUCKET}/{PREFIX}")

    # Parquet “part-” files (like in your screenshot)
    df = read_parquet_keys_to_df(BUCKET, keys)

    print("\n--- Data Preview ---")
    print(df.head(10))
    print("\n--- Schema / dtypes ---")
    print(df.dtypes)
    print("\n--- Row count ---")
    print(len(df))

    # Optional: save locally
    df.to_csv("day10_run_summary.csv", index=False)
    print("\nSaved: day10_run_summary.csv")

if __name__ == "__main__":
    main()


Found 4 objects under s3://new-demo-bucket-sumenth-12/audit/day10_run_summary/
Reading: s3://new-demo-bucket-sumenth-12/audit/day10_run_summary/part-00000-ea61f14c-3021-4ab8-a7df-c322ea8ec547-c000.snappy.parquet
Reading: s3://new-demo-bucket-sumenth-12/audit/day10_run_summary/part-00003-cfebb071-4df2-4cb7-83e4-8b283cc46284-c000.snappy.parquet

--- Data Preview ---
                              audit_history_out_path  \
0  s3://new-demo-bucket-sumenth-12/audit/day10_de...   

                                  curated_trips_path  \
0  s3://new-demo-bucket-sumenth-12/curated_data/y...   

                            job_name  \
0  day10-mdm-lifecycle-audit-orphans   

                             lifecycle_snapshot_path  \
0  s3://new-demo-bucket-sumenth-12/governance/lif...   

                                   master_zones_path  orphans_count  \
0  s3://new-demo-bucket-sumenth-12/master_data/ta...            384   

                                    orphans_out_path        generated_

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,2,2025-08-01 00:52:23,2025-08-01 01:12:20,1.0,8.44,1.0,N,138,141,1,33.8,6.0,0.5,5.00,6.94,1.0,57.49,2.5,1.75,0.00
1,2,2025-08-01 00:03:01,2025-08-01 00:15:33,2.0,4.98,1.0,N,138,193,1,21.2,6.0,0.5,0.00,0.00,1.0,30.45,0.0,1.75,0.00
2,7,2025-08-01 00:24:38,2025-08-01 00:24:38,2.0,1.89,1.0,N,249,45,1,14.2,0.0,0.5,3.99,0.00,1.0,23.94,2.5,0.00,0.75
3,7,2025-08-01 00:48:19,2025-08-01 00:48:19,1.0,2.35,1.0,N,79,229,1,11.4,0.0,0.5,3.43,0.00,1.0,20.58,2.5,0.00,0.75
4,2,2025-08-01 00:25:34,2025-08-01 00:33:18,1.0,2.14,1.0,N,43,48,1,11.4,1.0,0.5,2.57,0.00,1.0,19.72,2.5,0.00,0.75


In [5]:
import boto3
from io import BytesIO

BUCKET = "new-demo-bucket-sumenth-12"
DELTA_LOG_PREFIX = "audit/day10_run_summary/_delta_log/"

session = boto3.Session()
s3 = session.client("s3")

def list_delta_json(bucket, prefix):
    keys = []
    paginator = s3.get_paginator("list_objects_v2")
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get("Contents", []):
            k = obj["Key"]
            if k.endswith(".json"):
                keys.append(k)
    return sorted(keys)

keys = list_delta_json(BUCKET, DELTA_LOG_PREFIX)
print("Delta JSON files:", keys[-5:])

if keys:
    latest = keys[-1]
    print("Latest:", latest)

    bio = BytesIO()
    s3.download_fileobj(BUCKET, latest, bio)
    bio.seek(0)

    # Delta log JSON is newline-delimited JSON (one action per line)
    text = bio.read().decode("utf-8", errors="replace")
    print("\n--- First 30 lines of latest delta log ---")
    for i, line in enumerate(text.splitlines()[:30], 1):
        print(f"{i:02d}: {line}")


Delta JSON files: ['audit/day10_run_summary/_delta_log/00000000000000000000.json']
Latest: audit/day10_run_summary/_delta_log/00000000000000000000.json

--- First 30 lines of latest delta log ---
01: {"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
02: {"metaData":{"id":"728e22c9-733e-4596-9b41-99c56ed592d0","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"audit_history_out_path\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"curated_trips_path\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"job_name\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"lifecycle_snapshot_path\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"master_zones_path\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"orphans_count\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}},{\"name\":\"orphans_out_path\",\"type\":\"string\",\"nullable\":true,

In [6]:
import boto3
import json

# -------- CONFIG --------
BUCKET_NAME = "new-demo-bucket-sumenth-12"
OBJECT_KEY = "audit/day9_mdm/part-00000-5b8428dd-25c3-4a6d-8005-27255b9904c9-c000.json"
AWS_REGION = "us-east-1"   # change if needed
# ------------------------

# Create S3 client (uses configured AWS credentials)
s3 = boto3.client("s3", region_name=AWS_REGION)

# Read the object
response = s3.get_object(Bucket=BUCKET_NAME, Key=OBJECT_KEY)

# Read and decode content
content = response["Body"].read().decode("utf-8")

# If file is JSON (or JSON lines)
try:
    data = json.loads(content)
    print("=== JSON CONTENT ===")
    print(json.dumps(data, indent=2))
except json.JSONDecodeError:
    print("=== RAW FILE CONTENT (JSON Lines or text) ===")
    print(content)


=== JSON CONTENT ===
{
  "total_rows": 265,
  "auto_merge_rows": 265,
  "steward_review_rows": 0,
  "reject_rows": 0,
  "avg_match_confidence": 0.9896226415094374,
  "high_conf_threshold": 0.95,
  "med_conf_threshold": 0.8,
  "raw_zones_path": "s3://new-demo-bucket-sumenth-12/raw_data/reference",
  "master_out_path": "s3://new-demo-bucket-sumenth-12/master_data/taxi_zones_delta",
  "job_name": "day9-mdm-matching-dedup",
  "generated_at_utc": "2026-01-06T02:28:06.427Z"
}


In [7]:
records = [json.loads(line) for line in content.splitlines() if line.strip()]
print(f"Total records: {len(records)}")
print(json.dumps(records[:5], indent=2))  # preview first 5



Total records: 1
[
  {
    "total_rows": 265,
    "auto_merge_rows": 265,
    "steward_review_rows": 0,
    "reject_rows": 0,
    "avg_match_confidence": 0.9896226415094374,
    "high_conf_threshold": 0.95,
    "med_conf_threshold": 0.8,
    "raw_zones_path": "s3://new-demo-bucket-sumenth-12/raw_data/reference",
    "master_out_path": "s3://new-demo-bucket-sumenth-12/master_data/taxi_zones_delta",
    "job_name": "day9-mdm-matching-dedup",
    "generated_at_utc": "2026-01-06T02:28:06.427Z"
  }
]


In [8]:
import boto3
import pandas as pd

BUCKET = "new-demo-bucket-sumenth-12"
PREFIX  = "audit/day10_run_summary/"  # your folder

s3 = boto3.client("s3")

def list_prefix(bucket, prefix, max_keys=200):
    resp = s3.list_objects_v2(Bucket=bucket, Prefix=prefix, MaxKeys=max_keys)
    keys = [o["Key"] for o in resp.get("Contents", [])]
    return keys

# 1) PROOF: show delta table structure (_delta_log + parquet parts)
keys = list_prefix(BUCKET, PREFIX)
print("=== Objects under:", f"s3://{BUCKET}/{PREFIX}", "===")
for k in keys:
    print(k)

# 2) Find parquet parts (run summary data)
parquet_keys = [k for k in keys if k.endswith(".parquet") and "part-" in k]
if not parquet_keys:
    raise RuntimeError("No parquet part files found under the prefix.")

print("\n=== Parquet part files ===")
for k in parquet_keys:
    print(k)

# 3) Read parquet directly from S3 into pandas (best for screenshot)
# Uses pandas+pyarrow+s3fs under the hood
s3_paths = [f"s3://{BUCKET}/{k}" for k in parquet_keys]

df = pd.concat([pd.read_parquet(p) for p in s3_paths], ignore_index=True)

print("\n=== Run summary preview (for screenshot) ===")
print(df.head(50))

print("\n=== Columns ===")
print(list(df.columns))


=== Objects under: s3://new-demo-bucket-sumenth-12/audit/day10_run_summary/ ===
audit/day10_run_summary/_delta_log/00000000000000000000.json
audit/day10_run_summary/_delta_log_$folder$
audit/day10_run_summary/part-00000-ea61f14c-3021-4ab8-a7df-c322ea8ec547-c000.snappy.parquet
audit/day10_run_summary/part-00003-cfebb071-4df2-4cb7-83e4-8b283cc46284-c000.snappy.parquet

=== Parquet part files ===
audit/day10_run_summary/part-00000-ea61f14c-3021-4ab8-a7df-c322ea8ec547-c000.snappy.parquet
audit/day10_run_summary/part-00003-cfebb071-4df2-4cb7-83e4-8b283cc46284-c000.snappy.parquet

=== Run summary preview (for screenshot) ===
                              audit_history_out_path  \
0  s3://new-demo-bucket-sumenth-12/audit/day10_de...   

                                  curated_trips_path  \
0  s3://new-demo-bucket-sumenth-12/curated_data/y...   

                            job_name  \
0  day10-mdm-lifecycle-audit-orphans   

                             lifecycle_snapshot_path  \
0  s3://ne

In [9]:
df.head()

,audit_history_out_path,curated_trips_path,job_name,lifecycle_snapshot_path,master_zones_path,orphans_count,orphans_out_path,generated_at_utc
0,s3://new-demo-bucket-sumenth-12/audit/day10_de...,s3://new-demo-bucket-sumenth-12/curated_data/y...,day10-mdm-lifecycle-audit-orphans,s3://new-demo-bucket-sumenth-12/governance/lif...,s3://new-demo-bucket-sumenth-12/master_data/ta...,384,s3://new-demo-bucket-sumenth-12/governance/orp...,2026-01-06 03:00:53.841


In [10]:
import boto3, json

BUCKET = "new-demo-bucket-sumenth-12"
DELTA_LOG_PREFIX = "audit/day10_run_summary/_delta_log/"

s3 = boto3.client("s3")

resp = s3.list_objects_v2(Bucket=BUCKET, Prefix=DELTA_LOG_PREFIX)
log_files = sorted([o["Key"] for o in resp.get("Contents", [])])

# Pick the newest JSON commit file (ignore checkpoints)
json_logs = [k for k in log_files if k.endswith(".json")]
if not json_logs:
    raise RuntimeError("No Delta JSON log files found in _delta_log/")

latest = json_logs[-1]
print("Latest delta log file:", f"s3://{BUCKET}/{latest}")

obj = s3.get_object(Bucket=BUCKET, Key=latest)
lines = obj["Body"].read().decode("utf-8").splitlines()

# Print a few log actions (commitInfo / add / remove etc.)
print("\n=== Sample delta log actions (for screenshot) ===")
for i, line in enumerate(lines[:30]):
    try:
        print(json.loads(line))
    except:
        print(line)


Latest delta log file: s3://new-demo-bucket-sumenth-12/audit/day10_run_summary/_delta_log/00000000000000000000.json

=== Sample delta log actions (for screenshot) ===
{'protocol': {'minReaderVersion': 1, 'minWriterVersion': 2}}
{'metaData': {'id': '728e22c9-733e-4596-9b41-99c56ed592d0', 'format': {'provider': 'parquet', 'options': {}}, 'schemaString': '{"type":"struct","fields":[{"name":"audit_history_out_path","type":"string","nullable":true,"metadata":{}},{"name":"curated_trips_path","type":"string","nullable":true,"metadata":{}},{"name":"job_name","type":"string","nullable":true,"metadata":{}},{"name":"lifecycle_snapshot_path","type":"string","nullable":true,"metadata":{}},{"name":"master_zones_path","type":"string","nullable":true,"metadata":{}},{"name":"orphans_count","type":"long","nullable":true,"metadata":{}},{"name":"orphans_out_path","type":"string","nullable":true,"metadata":{}},{"name":"generated_at_utc","type":"timestamp","nullable":true,"metadata":{}}]}', 'partitionColumn

Latest delta log file: s3://new-demo-bucket-sumenth-12/audit/day10_run_summary/_delta_log/00000000000000000000.json

=== Sample delta log actions (for screenshot) ===
{'protocol': {'minReaderVersion': 1, 'minWriterVersion': 2}}
{'metaData': {'id': '728e22c9-733e-4596-9b41-99c56ed592d0', 'format': {'provider': 'parquet', 'options': {}}, 'schemaString': '{"type":"struct","fields":[{"name":"audit_history_out_path","type":"string","nullable":true,"metadata":{}},{"name":"curated_trips_path","type":"string","nullable":true,"metadata":{}},{"name":"job_name","type":"string","nullable":true,"metadata":{}},{"name":"lifecycle_snapshot_path","type":"string","nullable":true,"metadata":{}},{"name":"master_zones_path","type":"string","nullable":true,"metadata":{}},{"name":"orphans_count","type":"long","nullable":true,"metadata":{}},{"name":"orphans_out_path","type":"string","nullable":true,"metadata":{}},{"name":"generated_at_utc","type":"timestamp","nullable":true,"metadata":{}}]}', 'partitionColumn

In [12]:

df = pd.read_parquet(
    "s3://new-demo-bucket-sumenth-12/steward_queue/taxi_zones/"
)
print(df.head(20))


Empty DataFrame
Columns: [LocationID, Borough, Zone, service_zone, zone_norm, borough_norm, match_key, dupe_count, canonical_zone_norm, name_similarity, match_confidence, decision, mdm_state, submitted_at_utc, review_reason]
Index: []
